<a href="https://colab.research.google.com/github/YMMM98/Mask-R-CNN-Training-to-deployement-in-Colab/blob/main/Training_your_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We clone the original Mask R-CNN repo first


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN

# Checking GPU and installing dependencies


Don't forget to switch to use gpu.     Runtime -> Change runtime -> GPU

In [ ]:
!nvidia-smi

In [ ]:
%%shell 
pip install tensorflow==1.13.1 keras==2.0.8 
pip install h5py==2.10.0
pip install tensorflow-gpu==1.13.1

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
#we need true

# Setting up the config file

In [ ]:
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw

In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN git repo
ROOT_DIR = '/content/Mask_RCNN'
assert os.path.exists(ROOT_DIR)

# Import mrcnn libraries
sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

You can download your own pre-trained model and continue training or use the COCO or ImageNet weights

In [1]:
%%shell
gdown https://drive.google.com/uc?id=1Pw5iDiIFc6_HL9E6kXU0fkw-wVybqbNw
#I download my own pre-trained model from google drive

Downloading...
From: https://drive.google.com/uc?id=1Pw5iDiIFc6_HL9E6kXU0fkw-wVybqbNw
To: /content/mask_rcnn_rod101_0004.h5
100% 256M/256M [00:05<00:00, 48.4MB/s]


In [ ]:
# Or Download COCO weights
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
   utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class morsnowConfig(Config):
    """Configuration for training on the Rod dataset.
    Derives from the base Config class and overrides values specific
    to ours.
    """
    # Config name
    NAME = "Rod101"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    # Number of images to train with on each GPU. A 12GB GPU can typically
    # handle 2 images of 1024x1024px.
    # Adjust based on the GPU memory and image sizes you got in colab.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 (Snow Rod)

    # I chose 1024x1024 as most of our pictures are 1200x1600
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    # learning rate and momentum
    LEARNING_RATE = 0.001
    LEARNING_MOMENTUM = 0.9    

    # Weight decay regularization
    WEIGHT_DECAY = 0.0001

    # Number of training steps per epoch
    # This doesn't need to match the size of the training set. Tensorboard
    # updates are saved at the end of each epoch, so setting this to a
    # smaller number means getting more frequent TensorBoard updates.
    # Validation stats are also calculated at each epoch end and they
    # might take a while, so don't set this too small to avoid spending
    # a lot of time on validation stats.
    STEPS_PER_EPOCH = 45

    # This is how often validation is run. 
    # the bigger, the better the accuracy and the slower the training
    VALIDATION_STEPS = 5
    
    # the availabe ones for now are resnet50 and resnet101 in this repo.
    BACKBONE = 'resnet101'

    # Other hyperparameters, no need to modify them as the performance is already good enough
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
    #Check Mask_RCNN/mrcnn/config.py for more parameters to tune.

config = morsnowConfig()
config.display()

# Pre-processing the Dataset


We upload our zip file, that contains 4 things :  

1.  Training folder
2.  Validation folder
3.  Annotations for training  (Coco Format)
4.  Annotations for Validation

i used Makesense.ai for the annotation, but VGG annotator is good too.

In [ ]:
!unzip /content/dataset.zip

Helper class to prepare the dataset

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ See http://cocodataset.org/#home for more information. """
    def load_data(self, annotation_json, images_dir):
        
        """ Load the coco-format dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

In [ ]:
dataset_train = CocoLikeDataset()
dataset_train.load_data( '/content/cocoTRAIN.json' , '/content/Train')
dataset_train.prepare()

dataset_val = CocoLikeDataset()
dataset_val.load_data('/content/cocoVAL.json', '/content/val')
dataset_val.prepare()

Visualizing the dataset

In [ ]:
dataset = dataset_train
image_ids = np.random.choice(dataset.image_ids, 4)
for image_id in image_ids:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)

# Training

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

If you downloaded the coco weights, run this

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last if we upload our own weights

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [ ]:
# Train the head branches Passing layers="heads" freezes all layers except the head layers. You can also pass a regular expression to select
# which layers to train by name pattern.
# or layers = "all" if you want to fine-tune all layers
start_train = time.time()

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE,
            epochs=3, 
            layers='heads')

end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')

# Inspecting the results


We can use tensorboad to check the loss

In [ ]:
%load_ext tensorboard.notebook
#%reload_ext tensorboard.notebook

In [ ]:
event = /content/Mask_RCNN/logs/rod10120220509T2129
# put the path that leads to the directory that has the tf.events file, should be in mrcnn> logs > NameOfyourConfig

In [ ]:
%tensorboard --logdir event

  I still need to add mAP evaluation, weights histogram, precision/recall,...

# Testing the model

In [ ]:
class InferenceConfig(morsnowConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    #minimum confidence
    DETECTION_MIN_CONFIDENCE = 0.8
    

inference_config = InferenceConfig()

Recreate the model in inference mode

In [ ]:
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

Give the path to saved weights

In [ ]:
# Get path to saved weights 
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()
model.load_weights(model_path, by_name=True)

Or use your downloanded pre-trained weights

In [ ]:
model_path = '/content/mask_rcnn_rod101_0004.h5'
model.load_weights(model_path, by_name=True)

Upload some images to test.

In [ ]:
!unzip /content/test.zip

In [ ]:
#background + our classes
class_names = ['BG','Rod']

Run prediction on all your folder images

In [ ]:
import skimage
real_test_dir = '/content/Real_test'
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], figsize=(5,5))

Run prediction on one image

In [ ]:
path = '/content/test.jpg'
image = skimage.io.imread(path)

# Run detection
results = model.detect([image], verbose=0)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])